# 将骑手轨迹中的基站序号对应上点

In [25]:
import pandas as pd
# 读取两个表
df_traj = pd.read_csv("外卖员每小时对应基站_基站编码.csv")
df_bs = pd.read_csv("新业态基站_编码(1).csv")

        cgi_id        lon_gz        lat_gz
0       110082  29557.547815  64409.446807
1        84781  74146.816341  28973.560021
2       119413  48669.372120  28968.959635
3        80588  37641.533104  29347.618910
4        89256  33771.694469  35405.831870
...        ...           ...           ...
119566   90293  55895.487750  56283.125792
119567   64757  45514.156231  25227.404303
119568   72060  48016.380970  25768.189958
119569   19844  38500.781976  27613.202501
119570   18951  35231.333957  25885.142344

[119571 rows x 3 columns]


/var/folders/fm/w8wfgmlj0c51rhq3ddxsbsch0000gn/T/ipykernel_42062/1934438792.py:4: DtypeWarning: Columns (1,2,3,4,5,7,8,10,11,13,14,15,16,19,20,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_traj = pd.read_csv("外卖员每小时对应基站_基站编码.csv")


In [26]:
# 宽表转成长表
df_long = df_traj.melt(id_vars=["UID"],
                       var_name="hour",
                       value_name="cgi_id")

# 提取小时编号
df_long["hour"] = df_long["hour"].str.replace("cgi_", "").astype(int)

# 修复关键点：统一 cgi_id 格式
df_long["cgi_id"] = pd.to_numeric(df_long["cgi_id"], errors="coerce").dropna().astype("Int64").astype(str)
df_bs["cgi_id"]   = df_bs["cgi_id"].astype(str)

# 合并基站坐标
df_long = df_long.merge(df_bs, on="cgi_id", how="left")


In [27]:
# 按快递员编号 UID 排序
df_long = df_long.sort_values(by=["UID", "hour"]).reset_index(drop=True)
# 保存结果
df_long.to_csv("外卖员轨迹点表.csv", index=False)